In [7]:
from ase import Atoms
from ase.build import bulk

In [8]:
from ase import Atoms
from ase.build import bulk
import numpy as np
from pyevtk.hl import pointsToVTK
from pyevtk.hl import gridToVTK#, pointsToVTKAsTIN


In [3]:
# Define the aluminum supercell
aluminum = bulk('Al', 'fcc', a=4.05, cubic=True)
supercell = aluminum.repeat((150, 150, 150))  # Repeat the basic cell to form a supercell

In [8]:
supercell.get_atomic_numbers()

array([13, 13, 13, ..., 13, 13, 13])

In [9]:
supercell.get_positions()

array([[  0.   ,   0.   ,   0.   ],
       [  0.   ,   2.025,   2.025],
       [  2.025,   0.   ,   2.025],
       ...,
       [603.45 , 605.475, 605.475],
       [605.475, 603.45 , 605.475],
       [605.475, 605.475, 603.45 ]])

In [10]:
def is_inside_halftorus(position, center, major_radius, minor_radius):
    displacement = position - center
    r_cylindrical = np.linalg.norm(displacement[:2])
    is_in_torus = (r_cylindrical - major_radius)**2 + displacement[2]**2 < minor_radius**2
    return is_in_torus and displacement[0] > 0  # for half torus


def is_inside_sphere(position, center, radius):
    displacement = position - center
    return np.linalg.norm(displacement) < radius



In [11]:
center_torus = supercell.cell.diagonal() / 2
major_radius = 100  # Adjust based on your requirements
minor_radius = 30   # Adjust based on your requirements

center_sphere1 = center_torus - [0,major_radius, 0]  # Adjust based on your torus orientation
center_sphere2 = center_torus + [0,major_radius,  0]  # Adjust based on your torus orientation

for atom in supercell:
    if (is_inside_halftorus(atom.position, center_torus, major_radius, minor_radius) or 
        is_inside_sphere(atom.position, center_sphere1, minor_radius) or
        is_inside_sphere(atom.position, center_sphere2, minor_radius)):
        atom.symbol = 'Cu'


In [12]:
center_torus

array([303.75, 303.75, 303.75])

In [13]:
supercell.get_positions()

array([[  0.   ,   0.   ,   0.   ],
       [  0.   ,   2.025,   2.025],
       [  2.025,   0.   ,   2.025],
       ...,
       [603.45 , 605.475, 605.475],
       [605.475, 603.45 , 605.475],
       [605.475, 605.475, 603.45 ]])

In [15]:
from pyevtk.hl import pointsToVTK
from pyevtk.hl import gridToVTK#, pointsToVTKAsTIN
image = supercell.get_positions()
FILE_PATH1 = "20230615_dis_half_tourus"
print(FILE_PATH1)
x = np.ascontiguousarray(image[:,0])
y= np.ascontiguousarray(image[:,1])
z = np.ascontiguousarray(image[:,2])
label = np.ascontiguousarray( supercell.get_atomic_numbers())
pointsToVTK(FILE_PATH1,x,y,z, data = {"label" : label }  )

20230615_dis_half_tourus


'/Users/alaukiksaxena/tensorflow-test/Development_20230613/Dislocations_20230613/Synthetic_Data_Point_clouds/20230615_dis_half_tourus.vtu'

In [ ]:
# making a cylinder
"""import numpy as np

def is_inside_cylinder(position, center, axis, radius):
    displacement = position - center
    distance_to_axis = np.linalg.norm(displacement - np.dot(displacement, axis) * axis)
    return distance_to_axis < radius

center = supercell.cell.diagonal() / 2
axis = np.array([0, 0, 1])  # Change this to the desired axis of the cylinder
radius = 10  # Change this to the desired radius

for atom in supercell:
    if is_inside_cylinder(atom.position, center, axis, radius):
        atom.symbol = 'Cu'""";

#Making a torus

"""
def is_inside_torus(position, center, major_radius, minor_radius):
    displacement = position - center
    # Keep only x and y for cylindrical coordinates
    r_cylindrical = np.linalg.norm(displacement[:2])
    return (r_cylindrical - major_radius)**2 + displacement[2]**2 < minor_radius**2

center = supercell.cell.diagonal() / 2
major_radius = 100  # Adjust based on your requirements
minor_radius = 30 # Adjust based on your requirements

for atom in supercell:
    if is_inside_torus(atom.position, center, major_radius, minor_radius):
        atom.symbol = 'Cu'
        
"""